# Import libraries

In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
import joblib  
from statsmodels.tsa.arima.model import ARIMA



### Set file Name

In [215]:
file_name = 'clean_data'
data = pd.read_csv("../data/"+file_name+".csv")

In [216]:
data

,state,location,type,so2,no2,rspm,spm,date
0,Andhra Pradesh,Hyderabad,Residential,4.8,17.4,0.0,0.0,1990-02-01
1,Andhra Pradesh,Hyderabad,Industrial,3.1,7.0,0.0,0.0,1990-02-01
2,Andhra Pradesh,Hyderabad,Residential,6.2,28.5,0.0,0.0,1990-02-01
3,Andhra Pradesh,Hyderabad,Residential,6.3,14.7,0.0,0.0,1990-03-01
4,Andhra Pradesh,Hyderabad,Industrial,4.7,7.5,0.0,0.0,1990-03-01
...,...,...,...,...,...,...,...,...
435730,West Bengal,ULUBERIA,Residential,20.0,44.0,148.0,0.0,2015-12-15
435731,West Bengal,ULUBERIA,Residential,17.0,44.0,131.0,0.0,2015-12-18
435732,West Bengal,ULUBERIA,Residential,18.0,45.0,140.0,0.0,2015-12-21
435733,West Bengal,ULUBERIA,Residential,22.0,50.0,143.0,0.0,2015-12-24


# Calculate AQI

### AQI

In [217]:
def calculate_si(so2):
    si=0
    if (so2<=40):
     si= so2*(50/40)
    if (so2>40 and so2<=80):
     si= 50+(so2-40)*(50/40)
    if (so2>80 and so2<=380):
     si= 100+(so2-80)*(100/300)
    if (so2>380 and so2<=800):
     si= 200+(so2-380)*(100/800)
    if (so2>800 and so2<=1600):
     si= 300+(so2-800)*(100/800)
    if (so2>1600):
     si= 400+(so2-1600)*(100/800)
    return si
data['si']=data['so2'].apply(calculate_si)
df= data[['so2','si']]
df.head()

,so2,si
0,4.8,6.000
1,3.1,3.875
2,6.2,7.750
3,6.3,7.875
4,4.7,5.875


In [218]:
def calculate_ni(no2):
    ni=0
    if(no2<=40):
     ni= no2*50/40
    elif(no2>40 and no2<=80):
     ni= 50+(no2-14)*(50/40)
    elif(no2>80 and no2<=180):
     ni= 100+(no2-80)*(100/100)
    elif(no2>180 and no2<=280):
     ni= 200+(no2-180)*(100/100)
    elif(no2>280 and no2<=400):
     ni= 300+(no2-280)*(100/120)
    else:
     ni= 400+(no2-400)*(100/120)
    return ni
data['ni']=data['no2'].apply(calculate_ni)
df= data[['no2','ni']]
df.head()

,no2,ni
0,17.4,21.750
1,7.0,8.750
2,28.5,35.625
3,14.7,18.375
4,7.5,9.375


In [219]:
def calculate_(rspm):
    rpi=0
    if(rpi<=30):
     rpi=rpi*50/30
    elif(rpi>30 and rpi<=60):
     rpi=50+(rpi-30)*50/30
    elif(rpi>60 and rpi<=90):
     rpi=100+(rpi-60)*100/30
    elif(rpi>90 and rpi<=120):
     rpi=200+(rpi-90)*100/30
    elif(rpi>120 and rpi<=250):
     rpi=300+(rpi-120)*(100/130)
    else:
     rpi=400+(rpi-250)*(100/130)
    return rpi
data['rpi']=data['rspm'].apply(calculate_si)
df= data[['rspm','rpi']]
df.tail()

,rspm,rpi
435730,148.0,122.666667
435731,131.0,117.000000
435732,140.0,120.000000
435733,143.0,121.000000
435734,171.0,130.333333


In [220]:
def calculate_spi(spm):
    spi=0
    if(spm<=50):
     spi=spm
    if(spm<50 and spm<=100):
     spi=spm
    elif(spm>100 and spm<=250):
     spi= 100+(spm-100)*(100/150)
    elif(spm>250 and spm<=350):
     spi=200+(spm-250)
    elif(spm>350 and spm<=450):
     spi=300+(spm-350)*(100/80)
    else:
     spi=400+(spm-430)*(100/80)
    return spi
data['spi']=data['spm'].apply(calculate_spi)
df= data[['spm','spi']]
df.tail()
#many data values of rspm values is unawailable since it was not measure before

,spm,spi
435730,0.0,0.0
435731,0.0,0.0
435732,0.0,0.0
435733,0.0,0.0
435734,0.0,0.0


In [221]:
#function to calculate the air quality index (AQI) of every data value
#its is calculated as per indian govt standards
def calculate_aqi(si,ni,spi,rpi):
    aqi=0
    if(si>ni and si>spi and si>rpi):
     aqi=si
    if(spi>si and spi>ni and spi>rpi):
     aqi=spi
    if(ni>si and ni>spi and ni>rpi):
     aqi=ni
    if(rpi>si and rpi>ni and rpi>spi):
     aqi=rpi
    return aqi
data['AQI']=data.apply(lambda x:calculate_aqi(x['si'],x['ni'],x['spi'],x['rpi']),axis=1)
df= data[['date','state','si','ni','rpi','spi','AQI']]
df.head()

,date,state,si,ni,rpi,spi,AQI
0,1990-02-01,Andhra Pradesh,6.000,21.750,0.0,0.0,21.750
1,1990-02-01,Andhra Pradesh,3.875,8.750,0.0,0.0,8.750
2,1990-02-01,Andhra Pradesh,7.750,35.625,0.0,0.0,35.625
3,1990-03-01,Andhra Pradesh,7.875,18.375,0.0,0.0,18.375
4,1990-03-01,Andhra Pradesh,5.875,9.375,0.0,0.0,9.375


In [222]:
data.head()

,state,location,type,so2,no2,rspm,spm,date,si,ni,rpi,spi,AQI
0,Andhra Pradesh,Hyderabad,Residential,4.8,17.4,0.0,0.0,1990-02-01,6.000,21.750,0.0,0.0,21.750
1,Andhra Pradesh,Hyderabad,Industrial,3.1,7.0,0.0,0.0,1990-02-01,3.875,8.750,0.0,0.0,8.750
2,Andhra Pradesh,Hyderabad,Residential,6.2,28.5,0.0,0.0,1990-02-01,7.750,35.625,0.0,0.0,35.625
3,Andhra Pradesh,Hyderabad,Residential,6.3,14.7,0.0,0.0,1990-03-01,7.875,18.375,0.0,0.0,18.375
4,Andhra Pradesh,Hyderabad,Industrial,4.7,7.5,0.0,0.0,1990-03-01,5.875,9.375,0.0,0.0,9.375


In [224]:
fileTemp = data[['date','so2','no2','rspm','spm','si','ni','rpi','spi','AQI']]
fileTemp.to_csv('AQIraw_1.csv', index=False)

### save File

In [ ]:
# df = df.drop(columns=['state'])
# df.to_csv('../data/cal_'+file_name+'.csv', index=False)

In [ ]:
#skip
df["date"] = pd.to_datetime(df["date"])

# Group by year and month, calculate mean
monthly_mean = df.groupby(df["date"].dt.to_period("M")).mean()

# Drop the original 'date' column to avoid confusion (optional)
monthly_mean = monthly_mean.drop(columns=['date'])

# Move index to a column
monthly_mean.reset_index(inplace=True)

# Rename the index column to 'date'
monthly_mean.rename(columns={"date": "period"}, inplace=True)  # To avoid confusion, keep the Period type as "period"

# Convert 'period' to datetime (optional)
monthly_mean['date'] = monthly_mean['period'].dt.to_timestamp()

monthly_mean = monthly_mean.drop(columns=['date'])

# Show the first few rows to verify
print(monthly_mean.head())

monthly_mean.to_csv('../data/cal_'+file_name+'.csv', index=False)

# Visualization

In [ ]:
#skip
file_path = "../data/cal_"+file_name+".csv" 
data = pd.read_csv(file_path)
data

FileNotFoundError: [Errno 2] No such file or directory: '../data/cal_clean_data.csv'

In [ ]:
data.describe()

,si,ni,rpi,spi,AQI
count,430345.000000,430345.000000,430345.000000,430345.000000,430345.000000
mean,12.280945,34.844651,90.093113,81.751245,142.782761
std,12.332569,29.869610,44.571472,145.693134,117.529664
min,0.000000,0.000000,0.000000,-75.000000,0.000000
25%,5.000000,16.250000,60.000000,0.000000,80.000000
50%,8.875000,26.250000,101.333333,0.000000,115.333333
75%,16.250000,40.000000,118.666667,147.333333,159.000000
max,313.625000,796.666667,988.379167,3125.000000,3125.000000


In [ ]:
import pandas as pd

# # Sample DataFrame (replace this with your actual dataset)
# data = {
#     'date': ['1990-02-01', '1990-02-01', '1990-02-01', '1990-03-01', '1990-03-01'],
#     'si': [6.0, 3.875, 7.75, 7.875, 5.875],
#     'ni': [21.75, 8.75, 35.625, 18.375, 9.375],
#     'rpi': [0.0, 0.0, 0.0, 0.0, 0.0],
#     'spi': [0.0, 0.0, 0.0, 0.0, 0.0],
#     'AQI': [21.75, 8.75, 35.625, 18.375, 9.375]
# }

df = data.copy()

# Convert the 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'])

# Extract year and month from the 'date' column
df['year_month'] = df['date'].dt.to_period('M')

# Group by the 'year_month' column and calculate mean and standard deviation for each feature
monthly_stats = df.groupby('year_month').agg(
    si_mean=('si', 'mean'), si_sd=('si', 'std'),
    ni_mean=('ni', 'mean'), ni_sd=('ni', 'std'),
    rpi_mean=('rpi', 'mean'), rpi_sd=('rpi', 'std'),
    spi_mean=('spi', 'mean'), spi_sd=('spi', 'std'),
    AQI_mean=('AQI', 'mean'), AQI_sd=('AQI', 'std')
).reset_index()

print(monthly_stats)


    year_month    si_mean      si_sd    ni_mean      ni_sd    rpi_mean  \
0      1987-01  22.348140  18.375169  37.779175  32.301903  109.610928   
1      1987-02  24.876742  23.597709  44.821294  41.966442  109.610928   
2      1987-03  24.794715  23.335744  43.571985  61.991872  109.610928   
3      1987-04  23.476983  16.252608  35.470529  33.231484  109.610928   
4      1987-05  24.704336  22.356301  49.439325  39.607540  109.610928   
..         ...        ...        ...        ...        ...         ...   
341    2015-08   9.608154   8.667738  27.655081  20.991727   80.995864   
342    2015-09   9.634629   8.852012  29.900307  22.799051   87.495852   
343    2015-10  10.470647   9.121724  33.793137  27.664357   96.163273   
344    2015-11  11.093269  10.464518  37.395871  31.356350  102.797323   
345    2015-12  11.278626  11.127465  38.671292  32.104524  105.974409   

        rpi_sd    spi_mean      spi_sd    AQI_mean      AQI_sd  
0     0.000000  250.052355  225.432154  259.56

In [ ]:
monthly_stats.to_csv('temp.csv',index=False)

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np  # Import numpy for log transformation

df = data.copy()

# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Extract year and month and convert to string
df['year_month'] = df['date'].dt.to_period('M').astype(str)

# Apply log transformation to the features
features = ['si', 'ni', 'rpi', 'spi', 'AQI']
for feature in features:
    # Log transform the feature (log1p for handling zero values)
    df[feature] = np.log1p(df[feature])  # Use log1p to handle zero and negative values

# Plot the boxplot for each feature per month
for feature in features:
    fig = px.box(df, x='year_month', y=feature, title=f'Boxplot of Log-transformed {feature} by Month')
    fig.show()


In [ ]:
data.head()

,period,si,ni,rpi,spi,AQI
0,1987-01,22.348140,37.779175,109.610928,250.052355,259.563812
1,1987-02,24.876742,44.821294,109.610928,239.155179,253.636467
2,1987-03,24.794715,43.571985,109.610928,306.273945,322.938366
3,1987-04,23.476983,35.470529,109.610928,195.811594,227.085685
4,1987-05,24.704336,49.439325,109.610928,315.872807,336.653480


In [ ]:
import pandas as pd
import numpy as np  # Import numpy for log transformation

# Assuming 'data' is your original DataFrame
df = data.copy()

# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Apply log(x + 1) transformation to the features
features = ['si', 'ni', 'rpi', 'spi', 'AQI']
for feature in features:
    df[feature] = np.log1p(df[feature])  # Use log(x + 1) to handle zero and small values

# Save the transformed DataFrame to a CSV file
df.to_csv('normalized_data_log1p.csv', index=False)

# Optionally, save to an Excel file
# df.to_excel('normalized_data_log1p.xlsx', index=False)


e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log1p



In [ ]:
import plotly.express as px
file_path = f"../data/cal_{file_name}.csv" 
data = pd.read_csv(file_path)

data['period'] = pd.to_datetime(data['period'])
data

,period,si,ni,rpi,spi,AQI
0,1987-01-01,26.037112,41.666060,109.610928,264.903631,274.388053
1,1987-02-01,31.748164,57.736384,109.610928,257.680521,271.554676
2,1987-03-01,29.469050,52.919392,109.610928,331.511119,339.183486
3,1987-04-01,26.106093,49.351651,109.610928,245.386905,263.349180
4,1987-05-01,33.128343,63.969275,109.610928,322.787879,338.320690
...,...,...,...,...,...,...
340,2015-08-01,11.436307,31.634363,85.881268,180.522320,180.538852
341,2015-09-01,11.596945,34.892686,92.875167,180.522320,180.612748
342,2015-10-01,12.249911,37.479814,100.210440,180.522320,180.644813
343,2015-11-01,12.887695,40.721222,103.499169,180.522320,181.100704


In [ ]:

# Create the line graph
fig = px.line(
    data, 
    x='period', 
    y='AQI', 
    title="AQI Over Time", 
    labels={'period': 'Time Period', 'AQI': 'Air Quality Index'},
    template='plotly_white'
)

# Show the graph
fig.show()

In [ ]:
import pandas as pd

df = pd.read_csv('normalized_data_log1p.csv')

# Convert the 'date' column to datetime format
df["date"] = pd.to_datetime(df["date"])

# Select only numeric columns for grouping (exclude 'date' and any non-numeric columns)
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Group by year and month, calculating the mean for each month on numeric columns
monthly_mean = df.groupby(df["date"].dt.to_period("M"))[numeric_columns].median()

# Reset the index to move 'period' to a column
monthly_mean.reset_index(inplace=True)

# Rename the 'period' column to 'date'
monthly_mean.rename(columns={"date": "period"}, inplace=True)

# Convert the 'period' to a timestamp (optional)
monthly_mean['date'] = monthly_mean['period'].dt.to_timestamp()

# Drop the 'date' column since we now have the period in datetime format
monthly_mean = monthly_mean.drop(columns=['date'])

# Show the first few rows to verify the output
monthly_mean

# Save the result to a new CSV file
# monthly_mean.to_csv(f'../data/cal_{file_name}', index=False)


,period,so2,no2,rspm,spm,pm2_5,si,ni,rpi,spi,AQI
0,1987-01,12.900000,25.809623,108.832784,225.00000,40.791467,2.840539,3.504416,4.706019,5.238213,5.216746
1,1987-02,12.600000,25.809623,108.832784,220.89174,40.791467,2.818370,3.504416,4.706019,5.274590,5.201776
2,1987-03,13.700000,21.300000,108.832784,254.00000,40.791467,2.897292,3.318721,4.706019,5.476464,5.341856
3,1987-04,13.100000,25.809623,108.832784,209.00000,40.791467,2.855032,3.504416,4.706019,5.245268,5.157138
4,1987-05,10.829414,25.809623,108.832784,329.00000,40.791467,2.676681,3.504416,4.706019,5.864341,5.634790
...,...,...,...,...,...,...,...,...,...,...,...
341,2015-08,6.000000,18.000000,67.000000,220.78348,40.791467,2.140066,3.157000,4.439706,5.201379,5.201379
342,2015-09,6.000000,20.000000,73.000000,220.78348,40.791467,2.140066,3.258097,4.524502,5.201379,5.201379
343,2015-10,6.000000,20.000000,87.000000,220.78348,40.791467,2.140066,3.258097,4.637960,5.201379,5.201379
344,2015-11,7.000000,22.000000,98.000000,220.78348,40.791467,2.277267,3.349904,4.672829,5.201379,5.201379


In [ ]:
import plotly.express as px
import pandas as pd

# Assuming monthly_mean is already available as a DataFrame
data = monthly_mean

# Convert 'period' (Period type) to datetime (Timestamp type) for correct plotting
data['period'] = data['period'].dt.to_timestamp()

# Create the line graph for AQI over time
fig = px.line(
    data, 
    x='period', 
    y='AQI', 
    title="AQI Over Time", 
    labels={'period': 'Time Period', 'AQI': 'Air Quality Index'},
    template='plotly_white'
)

# Show the graph
fig.show()


In [228]:
df = pd.read_csv('AQIraw_1.csv')

In [229]:
df["date"] = pd.to_datetime(df["date"])

# Group by year and month, calculate mean
monthly_mean = df.groupby(df["date"].dt.to_period("M")).mean()

# Drop the original 'date' column to avoid confusion (optional)
monthly_mean = monthly_mean.drop(columns=['date'])

# Move index to a column
monthly_mean.reset_index(inplace=True)

# Rename the index column to 'date'
monthly_mean.rename(columns={"date": "period"}, inplace=True)  # To avoid confusion, keep the Period type as "period"

# Convert 'period' to datetime (optional)
monthly_mean['date'] = monthly_mean['period'].dt.to_timestamp()

monthly_mean = monthly_mean.drop(columns=['date'])

# Show the first few rows to verify
print(monthly_mean.head())

monthly_mean.to_csv('AQIMonthly_1.csv', index=False)

    period        so2        no2  rspm         spm         si         ni  rpi  \
0  1987-01  15.182979  20.858865   0.0  265.340426  18.500236  29.364184  0.0   
1  1987-02  17.693382  25.986765   0.0  259.750000  20.948652  36.680515  0.0   
2  1987-03  17.708772  31.340351   0.0  307.333333  21.848099  38.427632  0.0   
3  1987-04  15.956522  15.991304   0.0  225.173913  19.945652  27.054348  0.0   
4  1987-05  15.235000  28.135000   0.0  328.300000  19.043750  37.813750  0.0   

          spi         AQI  
0  238.133570  242.438652  
1  227.525123  235.787929  
2  286.692982  294.558772  
3  195.811594  202.012681  
4  302.379167  307.991667  


In [230]:
# split data for train/test
data = pd.read_csv('AQIMonthly_1.csv')
# data = monthly_mean.copy()

split_index = int(0.6 * len(monthly_mean))

train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Save the splits to new CSV files
train_data.to_csv("AQI_trainlog1p.csv",index=False)
test_data.to_csv("AQI_testlog1p.csv",index=False)
# train_data.to_csv("../data/train_"+file_name+".csv", index=False)
# test_data.to_csv("../data/test_"+file_name+".csv", index=False)


In [ ]:
train_data.head()

,period,so2,no2,rspm,spm,pm2_5,si,ni,rpi,spi,AQI
0,1987-01-01,12.900000,25.809623,108.832784,225.00000,40.791467,2.840539,3.504416,4.706019,5.238213,5.216746
1,1987-02-01,12.600000,25.809623,108.832784,220.89174,40.791467,2.818370,3.504416,4.706019,5.274590,5.201776
2,1987-03-01,13.700000,21.300000,108.832784,254.00000,40.791467,2.897292,3.318721,4.706019,5.476464,5.341856
3,1987-04-01,13.100000,25.809623,108.832784,209.00000,40.791467,2.855032,3.504416,4.706019,5.245268,5.157138
4,1987-05-01,10.829414,25.809623,108.832784,329.00000,40.791467,2.676681,3.504416,4.706019,5.864341,5.634790


# XGBoost Model

In [233]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import plotly.graph_objects as go
import joblib

# Load the training data
data = pd.read_csv('AQI_trainlog1p.csv', index_col=0)
data = data[['so2', 'no2', 'spm', 'rspm', 'AQI']]

# Reset index to get 'period' as a column if it's in the index
if 'period' not in data.columns:
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
data['period'] = pd.to_datetime(data['period'], format='%Y-%m')

# Create lag features
for lag in range(1, 2):  # Create lag features for 1 to 3 previous time steps
    data[f'AQI_lag_{lag}'] = data['AQI'].shift(lag)

# Drop rows with NaN values due to lagging
data = data.dropna()

# Features and target
features = [col for col in data.columns if col not in ['period', 'AQI']]
X = data[features]
y = data['AQI']

# Train-test split (use the last 40% of data as the test set)
train_size = int(len(data) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
scaler_path = '../results/minmax_scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Define the model
model = XGBRegressor(random_state=42)

# Define parameter grid
param_grid = {
    'n_estimators': [100,200,300,500],  # Number of trees
    'learning_rate': [0.1,0.01,0.05],  # Learning rate
    'max_depth': [5,6,7,8,9],  # Depth of each tree
    'subsample': [0.8],  # Fraction of samples to use
    'colsample_bytree': [1.0],  # Fraction of features to use
}

# Create a KFold instance without shuffling
kfold = KFold(n_splits=10, shuffle=False)

# Setup GridSearchCV with KFold
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=kfold,  # Use KFold with no shuffle
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Save the best model
model_path = '../results/xgboost-model.json'
best_model.save_model(model_path)
print(f"Model saved to {model_path}")

# Predictions on the test set
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"RMSE on test set(XGBoost): {rmse}")
print(f"MAE on test set(XGBoost): {mae}")
print(f"R² on test set(XGBoost): {r2}")

# Create a Plotly Figure
fig = go.Figure()

# Add train data
fig.add_trace(go.Scatter(x=data['period'][:train_size], y=y_train,
                         mode='lines', name='Train',
                         line=dict(color='blue')))

# Add test data
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_test,
                         mode='lines', name='Test',
                         line=dict(color='green')))

# Add predictions
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_pred,
                         mode='lines', name='Prediction',
                         line=dict(color='red', dash='dot')))

# Update layout
fig.update_layout(
    title='AQI Forecasting - Train, Test, and Predictions using XGBoost',
    xaxis_title='period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show metrics in the plot
fig.add_annotation(
    x=data['period'].iloc[-1],
    y=max(y_test),
    text=f"<b>RMSE:</b> {rmse:.2f}<br><b>MAE:</b> {mae:.2f}<br><b>R²:</b> {r2:.2f}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="black",
    borderwidth=1
)

# Show the plot
fig.show()


Scaler saved to ../results/minmax_scaler.pkl
Fitting 10 folds for each of 60 candidates, totalling 600 fits
Model saved to ../results/xgboost-model.json
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
RMSE on test set(XGBoost): 20.21252463117394
MAE on test set(XGBoost): 13.69075521875664
R² on test set(XGBoost): 0.7112957255167899


In [234]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import plotly.graph_objects as go
import joblib

# Load the new data
new_data = pd.read_csv('AQI_testlog1p.csv', index_col=0)
new_data = new_data[['so2', 'no2', 'spm', 'rspm', 'AQI']]

# Reset index to get 'period' as a column if it's in the index
if 'period' not in new_data.columns:
    new_data.reset_index(inplace=True)
    new_data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
new_data['period'] = pd.to_datetime(new_data['period'], format='%Y-%m')

# Create lag features for the new data (same as in the training data)
for lag in range(1, 2):
    new_data[f'AQI_lag_{lag}'] = new_data['AQI'].shift(lag)

# Drop rows with NaN values due to lagging (same as before)
new_data = new_data.dropna()

# Features for the new data (same as the training data)
features = [col for col in new_data.columns if col not in ['period', 'AQI']]
X_new = new_data[features]

# Load the saved scaler
scaler_path = '../results/minmax_scaler.pkl'
scaler = joblib.load(scaler_path)
print("Scaler loaded successfully")

# Scale the new data
X_new_scaled = scaler.transform(X_new)

# Load the trained model
model = XGBRegressor()
model.load_model('../results/xgboost-model.json')  # Replace with your saved model path

# Make predictions using the trained model
y_new_pred = model.predict(X_new_scaled)

# If you have the actual AQI values for the new data, you can compare them with the predictions
if 'AQI' in new_data.columns:
    y_new_actual = new_data['AQI']
    rmse_new = np.sqrt(mean_squared_error(y_new_actual, y_new_pred))
    mae_new = mean_absolute_error(y_new_actual, y_new_pred)
    r2_new = r2_score(y_new_actual, y_new_pred)
    
    print(f"RMSE on test set (XGBoost): {rmse_new}")
    print(f"MAE on test set (XGBoost): {mae_new}")
    print(f"R² on test set (XGBoost): {r2_new}")

# Optionally, visualize the results (if you want to compare predictions vs. actual)
fig_new = go.Figure()

# Add new data actual AQI (if available)
if 'AQI' in new_data.columns:
    fig_new.add_trace(go.Scatter(x=new_data['period'], y=new_data['AQI'],
                                mode='lines', name='Actual AQI',
                                line=dict(color='blue')))

# Add predictions
fig_new.add_trace(go.Scatter(x=new_data['period'], y=y_new_pred,
                            mode='lines', name='Predicted AQI',
                            line=dict(color='red', dash='dot')))

# Update layout
fig_new.update_layout(
    title='AQI Predictions using XGBoost',
    xaxis_title='Period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show the plot
fig_new.show()

Scaler loaded successfully
RMSE on test set (XGBoost): 31.447058561248504
MAE on test set (XGBoost): 27.70956581745768
R² on test set (XGBoost): 0.566113342116003


In [243]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import plotly.graph_objects as go
import joblib

# Load the training data
data = pd.read_csv('AQI_trainlog1p.csv', index_col=0)
data = data[['so2', 'no2', 'spm', 'rspm', 'AQI']]

# Reset index to get 'period' as a column if it's in the index
if 'period' not in data.columns:
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
data['period'] = pd.to_datetime(data['period'], format='%Y-%m')

# Create lag features (add lagged features for 1 to 5 previous time steps)
for lag in range(1, 3):  # Create lag features for 1 to 5 previous time steps
    data[f'AQI_lag_{lag}'] = data['AQI'].shift(lag)

# Drop rows with NaN values due to lagging
data = data.dropna()

# Features and target
features = [f'AQI_lag_{lag}' for lag in range(1, 3)]  # Use lagged AQI features (lag_1 to lag_5)
X = data[features]
y = data['AQI']

# Train-test split (use the last 30% of data as the test set)
train_size = int(len(data) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
scaler_path = '../results/minmax_scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Define the model
model = XGBRegressor(random_state=42)

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300, 500],  # Number of trees
    'learning_rate': [0.1, 0.01, 0.05],  # Learning rate
    'max_depth': [5, 6, 7, 8, 9],  # Depth of each tree
    'subsample': [0.8],  # Fraction of samples to use
    'colsample_bytree': [1.0],  # Fraction of features to use
}

# Create a KFold instance without shuffling
kfold = KFold(n_splits=10, shuffle=False)

# Setup GridSearchCV with KFold
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=kfold,  # Use KFold with no shuffle
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Save the best model
model_path = '../results/xgboost-model.json'
best_model.save_model(model_path)
print(f"Model saved to {model_path}")

# Predictions on the test set
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"RMSE on test set(XGBoost): {rmse}")
print(f"MAE on test set(XGBoost): {mae}")
print(f"R² on test set(XGBoost): {r2}")

# Create a Plotly Figure
fig = go.Figure()

# Add train data
fig.add_trace(go.Scatter(x=data['period'][:train_size], y=y_train,
                         mode='lines', name='Train',
                         line=dict(color='blue')))

# Add test data
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_test,
                         mode='lines', name='Test',
                         line=dict(color='green')))

# Add predictions
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_pred,
                         mode='lines', name='Prediction',
                         line=dict(color='red', dash='dot')))

# Update layout
fig.update_layout(
    title='AQI Forecasting - Train, Test, and Predictions using XGBoost',
    xaxis_title='period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show metrics in the plot
fig.add_annotation(
    x=data['period'].iloc[-1],
    y=max(y_test),
    text=f"<b>RMSE:</b> {rmse:.2f}<br><b>MAE:</b> {mae:.2f}<br><b>R²:</b> {r2:.2f}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="black",
    borderwidth=1
)

# Show the plot
fig.show()

Scaler saved to ../results/minmax_scaler.pkl
Fitting 10 folds for each of 60 candidates, totalling 600 fits
Model saved to ../results/xgboost-model.json
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
RMSE on test set(XGBoost): 38.476436053584315
MAE on test set(XGBoost): 30.37582846486406
R² on test set(XGBoost): -0.0461688041212156


In [ ]:
import pandas as pd

# Get feature importance
feature_importance = model.get_booster().get_score(importance_type='weight')

# Convert to DataFrame for better visualization
importance_df = pd.DataFrame(
    feature_importance.items(), columns=['Feature', 'Importance']
).sort_values(by='Importance', ascending=False)

# Print the feature importance
print(importance_df)


In [213]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import plotly.graph_objects as go
import joblib

# Load the new data
new_data = pd.read_csv('AQI_testlog1p.csv', index_col=0)
new_data = new_data[['so2', 'no2', 'spm', 'rspm', 'AQI']]

# Reset index to get 'period' as a column if it's in the index
if 'period' not in new_data.columns:
    new_data.reset_index(inplace=True)
    new_data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
new_data['period'] = pd.to_datetime(new_data['period'], format='%Y-%m')

# Create lag features for the new data (same as in the training data)
for lag in range(1, 2):
    new_data[f'AQI_lag_{lag}'] = new_data['AQI'].shift(lag)

# Drop rows with NaN values due to lagging (same as before)
new_data = new_data.dropna()

# Features for the new data (same as the training data)
features = [col for col in new_data.columns if col not in ['period', 'AQI']]
X_new = new_data[features]

# Load the saved scaler
scaler_path = '../results/minmax_scaler.pkl'
scaler = joblib.load(scaler_path)
print("Scaler loaded successfully")

# Scale the new data
X_new_scaled = scaler.transform(X_new)

# Load the trained model
model = XGBRegressor()
model.load_model('../results/xgboost-model.json')  # Replace with your saved model path

# Make predictions using the trained model
y_new_pred = model.predict(X_new_scaled)

# If you have the actual AQI values for the new data, you can compare them with the predictions
if 'AQI' in new_data.columns:
    y_new_actual = new_data['AQI']
    rmse_new = np.sqrt(mean_squared_error(y_new_actual, np.exp(y_new_pred)))
    mae_new = mean_absolute_error(y_new_actual, np.exp(y_new_pred))
    r2_new = r2_score(y_new_actual, np.exp(y_new_pred))
    
    print(f"RMSE on test set (XGBoost): {rmse_new}")
    print(f"MAE on test set (XGBoost): {mae_new}")
    print(f"R² on test set (XGBoost): {r2_new}")

# Optionally, visualize the results (if you want to compare predictions vs. actual)
fig_new = go.Figure()

# Add new data actual AQI (if available)
if 'AQI' in new_data.columns:
    fig_new.add_trace(go.Scatter(x=new_data['period'], y=new_data['AQI'],
                                mode='lines', name='Actual AQI',
                                line=dict(color='blue')))

# Add predictions
fig_new.add_trace(go.Scatter(x=new_data['period'], y=np.exp(y_new_pred),
                            mode='lines', name='Predicted AQI',
                            line=dict(color='red', dash='dot')))

# Update layout
fig_new.update_layout(
    title='AQI Predictions using XGBoost',
    xaxis_title='Period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show the plot
fig_new.show()

Scaler loaded successfully
RMSE on test set (XGBoost): 12.023945535124797
MAE on test set (XGBoost): 8.631573082453578
R² on test set (XGBoost): 0.6322993976319631


# Randomforest Model

In [ ]:
# Load the training data
data = pd.read_csv('../data/train_'+file_name+'.csv', index_col=0)

# Reset index to get 'period' as a column if it's in the index
if 'period' not in data.columns:
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
data['period'] = pd.to_datetime(data['period'], format='%Y-%m')

# Ensure the AQI column is numeric
data['AQI'] = pd.to_numeric(data['AQI'], errors='coerce')

# Drop rows with NaN values
data = data.dropna()

# Create lag features
data['AQI_lag_1'] = data['AQI'].shift(1)

# Drop rows with NaN values due to lagging
data = data.dropna()

# Features and target
features = [col for col in data.columns if col not in ['period', 'AQI']]
X = data[features]
y = data['AQI']

# Train-test split (use the last 40% of data as the test set)
train_size = int(len(data) * 0.6)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Initialize the RandomForestRegressor model
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
model_rf.fit(X_train, y_train)

# Save the model
joblib.dump(model_rf, '../results/random_forest_model.pkl')

# Predictions on the test set
y_pred = model_rf.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE on training set(Randomforest): {rmse}")
print(f"MAE on training set(Randomforest): {mae}")
print(f"R² on training set(Randomforest) : {r2}")

# Create a Plotly Figure
fig = go.Figure()

# Add train data
fig.add_trace(go.Scatter(x=data['period'][:train_size], y=y_train,
                         mode='lines', name='Train',
                         line=dict(color='blue')))

# Add test data
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_test,
                         mode='lines', name='Test',
                         line=dict(color='green')))

# Add predictions
fig.add_trace(go.Scatter(x=data['period'][train_size:], y=y_pred,
                         mode='lines', name='Prediction',
                         line=dict(color='red', dash='dot')))

# Update layout
fig.update_layout(
    title='AQI Forecasting - Train, Test, and Predictions using Random Forest',
    xaxis_title='period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show metrics in the plot
fig.add_annotation(
    x=data['period'].iloc[-1],
    y=max(y_test),
    text=f"<b>RMSE:</b> {rmse:.2f}<br><b>MAE:</b> {mae:.2f}<br><b>R²:</b> {r2:.2f}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="black",
    borderwidth=1
)

# Show the plot
fig.show()


RMSE on training set(Randomforest): 11.404596356512242
MAE on training set(Randomforest): 8.790075271751531
R² on training set(Randomforest) : 0.903512812805821


In [ ]:
# Load the new data (testing data)
new_data = pd.read_csv('../data/test_'+file_name+'.csv', index_col=0)

# Reset index to get 'period' as a column if it's in the index
if 'period' not in new_data.columns:
    new_data.reset_index(inplace=True)
    new_data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
new_data['period'] = pd.to_datetime(new_data['period'], format='%Y-%m')

# Set the 'period' column as the index
new_data.set_index('period', inplace=True)

# Ensure the AQI column is numeric (in case there are any issues with the data)
new_data['AQI'] = pd.to_numeric(new_data['AQI'], errors='coerce')

# Drop rows with NaN values (because of non-numeric AQI or missing values)
new_data = new_data.dropna()

# Create lag features for the new data (same as in the training data)
new_data['AQI_lag_1'] = new_data['AQI'].shift(1)

# Drop rows with NaN values due to lagging
new_data = new_data.dropna()

# Features for the model (use lag features and other potential features if available)
features = [col for col in new_data.columns if col not in ['AQI']]

# Define the features (X) and target (y) for testing the model
X_new = new_data[features]
y_new = new_data['AQI']

# Load the trained model from disk
model_rf = joblib.load('../results/random_forest_model.pkl')

# Make predictions on the new (testing) data
y_new_pred = model_rf.predict(X_new)

# If you have the actual AQI values for the new data, you can compare them with the predictions
# Assuming you have the 'AQI' column in 'new.csv' for evaluation
if 'AQI' in new_data.columns:
    y_new_actual = new_data['AQI']
    rmse_new = np.sqrt(mean_squared_error(y_new_actual, y_new_pred))
    mae_new = mean_absolute_error(y_new_actual, y_new_pred)
    r2_new = r2_score(y_new_actual, y_new_pred)
    
    print(f"RMSE on test set(Randomforest): {rmse_new}")
    print(f"MAE on test set(Randomforest): {mae_new}")
    print(f"R² on test set(Randomforest): {r2_new}")

# Optionally, visualize the results (if you want to compare predictions vs. actual)
fig_new = go.Figure()

# Add new data actual AQI (if available)
if 'AQI' in new_data.columns:
    fig_new.add_trace(go.Scatter(x=new_data.index, y=new_data['AQI'],
                                mode='lines', name='Actual AQI',
                                line=dict(color='blue')))

# Add predictions
fig_new.add_trace(go.Scatter(x=new_data.index, y=y_new_pred,
                            mode='lines', name='Predicted AQI',
                            line=dict(color='red', dash='dot')))

# Update layout
fig_new.update_layout(
    title='AQI Predictions using Random Forest',
    xaxis_title='Period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show the plot
fig_new.show()


RMSE on test set(Randomforest): 12.491516372775301
MAE on test set(Randomforest): 11.285676932961312
R² on test set(Randomforest): 0.7722825369856718


# ARIMA Model

In [ ]:
# Load the training data
data = pd.read_csv('../data/train_'+file_name+'.csv', index_col=0)

# Reset index to get 'period' as a column if it's in the index
if 'period' not in data.columns:
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
data['period'] = pd.to_datetime(data['period'], format='%Y-%m')

# Set period as the index for ARIMA
data.set_index('period', inplace=True)

# Define the target variable
y = data['AQI']

# Train-test split (use the last 20% of data as the test set)
train_size = int(len(data) * 0.6)
train, test = y[:train_size], y[train_size:]

# Fit ARIMA model (you can tune the p, d, q parameters as needed)
model = ARIMA(train, order=(3, 1, 2))  # Example order (p, d, q)
model_fit = model.fit()

# Predictions on the test set
y_pred = model_fit.forecast(steps=len(test))

# Evaluation Metrics
rmse = np.sqrt(mean_squared_error(test, y_pred))
mae = mean_absolute_error(test, y_pred)
r2 = r2_score(test, y_pred)

print(f"RMSE on training set(ARIMA): {rmse}")
print(f"MAE on training set(ARIMA): {mae}")
print(f"R² on training set(ARIMA): {r2}")

# Create a Plotly Figure
fig = go.Figure()

# Add train data
fig.add_trace(go.Scatter(x=data.index[:train_size], y=train,
                         mode='lines', name='Train',
                         line=dict(color='blue')))

# Add test data
fig.add_trace(go.Scatter(x=data.index[train_size:], y=test,
                         mode='lines', name='Test',
                         line=dict(color='green')))

# Add predictions
fig.add_trace(go.Scatter(x=data.index[train_size:], y=y_pred,
                         mode='lines', name='Prediction',
                         line=dict(color='red', dash='dot')))

# Update layout
fig.update_layout(
    title='AQI Forecasting - Train, Test, and Predictions using ARIMA',
    xaxis_title='Period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show metrics in the plot
fig.add_annotation(
    x=data.index[-1],
    y=max(test),
    text=f"<b>RMSE:</b> {rmse:.2f}<br><b>MAE:</b> {mae:.2f}<br><b>R²:</b> {r2:.2f}",
    showarrow=False,
    font=dict(size=12, color="black"),
    align="left",
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="black",
    borderwidth=1
)

# Show the plot
fig.show()

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning:

Non-stationary starting autoregressive parameters found. Using zero

RMSE on training set(ARIMA): 40.630588123499
MAE on training set(ARIMA): 34.51040656652901
R² on training set(ARIMA): -0.2246613891155005


e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [ ]:
new_data = pd.read_csv('../data/test_'+file_name+'.csv', index_col=0)

# Reset index to get 'period' as a column if it's in the index
if 'period' not in new_data.columns:
    new_data.reset_index(inplace=True)
    new_data.rename(columns={'index': 'period'}, inplace=True)

# Ensure the date column is in datetime format
new_data['period'] = pd.to_datetime(new_data['period'], format='%Y-%m')

# Set period as the index for ARIMA model
new_data.set_index('period', inplace=True)

# Define the target variable for the new data
y_new = new_data['AQI']

# Make predictions using the trained ARIMA model
y_new_pred = model_fit.forecast(steps=len(y_new))

# If you have the actual AQI values for the new data, you can compare them with the predictions
# Assuming you have the 'AQI' column in 'new_data.csv' for evaluation
if 'AQI' in new_data.columns:
    rmse_new = np.sqrt(mean_squared_error(y_new, y_new_pred))
    mae_new = mean_absolute_error(y_new, y_new_pred)
    r2_new = r2_score(y_new, y_new_pred)
    
    print(f"RMSE on test set(ARIMA): {rmse_new}")
    print(f"MAE on test set(ARIMA): {mae_new}")
    print(f"R² on test set(ARIMA): {r2_new}")

# Optionally, visualize the results (if you want to compare predictions vs. actual)
fig_new = go.Figure()

# Add new data actual AQI (if available)
if 'AQI' in new_data.columns:
    fig_new.add_trace(go.Scatter(x=new_data.index, y=new_data['AQI'],
                                mode='lines', name='Actual AQI',
                                line=dict(color='blue')))

# Add predictions
fig_new.add_trace(go.Scatter(x=new_data.index, y=y_new_pred,
                            mode='lines', name='Predicted AQI',
                            line=dict(color='red', dash='dot')))

# Update layout
fig_new.update_layout(
    title='AQI Predictions using ARIMA',
    xaxis_title='Period',
    yaxis_title='AQI',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

# Show the plot
fig_new.show()

RMSE on test set(ARIMA): 37.21397826703787
MAE on test set(ARIMA): 32.661253966037165
R² on test set(ARIMA): -1.0121227804620179


e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

e:\UniversityWorkY3\CPE312 Data Science\airindia\airindian\mlvenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

